In [1]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point,Polygon
import csv
import random
from geopandas.tools import sjoin
import multiprocessing as mp
import time

In [2]:
lat = []
lon = []
data_points = 100000
for i in range(data_points):
    x = random.uniform(-90,90)
    lat.append(x)
    
    
for i in range(data_points):
    x = random.uniform(-180,180)
    lon.append(x)

df_cord = pd.DataFrame(lon , columns = ['Lon'])
df_cord['Lat'] = lat
geometry = [Point(xy) for xy in zip(df_cord.Lon, df_cord.Lat)]
df_cord = df_cord.drop(['Lon' , 'Lat' ], axis=1)
crs = {'init': 'epsg:4326'}
gdf = geopandas.GeoDataFrame(df_cord, crs=crs, geometry=geometry)

In [3]:
print(gdf.shape)

(100000, 1)


In [4]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
# ax = world.plot( color='white', edgecolor='black' ,figsize = (40,30))
# gdf.plot(ax=ax, color='red')
# plt.title("World map showing location of given coordinate")
# print(world.head(3))
# plt.show()

In [5]:
df_new = world[['continent' , 'name' , 'geometry']]

In [6]:
def computation(data_split):
    for index,row in data_split.iterrows():
        filter = df_new['geometry'].contains(row['geometry'])
        x = df_new.loc[filter]
        value_c = x['name'].to_string(index = False)
        data_split.loc[index, 'Country'] = value_c if value_c != 'Series([], )' else np.NaN
    return data_split

In [7]:
#computation(gdf)

In [8]:
from pathos.multiprocessing import ProcessingPool as Pool

In [9]:
start_time = time.time()  

cores = 4 
partitions = cores


data_split = np.array_split(gdf, cores, axis=0)
pool = Pool(cores)
for i in range(len(data_split)):
    print(data_split[i].shape)
df_out = np.vstack(pool.map(computation, data_split))
pool.close()
pool.join()
pool.clear()
print("--- %s seconds ---" % (time.time() - start_time))

(25000, 1)
(25000, 1)
(25000, 1)
(25000, 1)
--- 331.8383090496063 seconds ---


In [10]:
op  = pd.DataFrame(data = df_out)
print(op)

                                                   0            1
0        POINT (123.2394736707416 -42.4249523865658)          NaN
1        POINT (170.9003061685207 74.06285613393979)          NaN
2      POINT (-40.74575591209535 -56.45632054314937)          NaN
3       POINT (-46.24473354841024 -17.9799401527677)       Brazil
4        POINT (163.2990348317372 18.71986630566725)          NaN
5        POINT (78.63527198947213 30.12335666727729)        India
6      POINT (-99.29762026096179 -76.25084126967425)   Antarctica
7        POINT (153.1774954311774 3.477128289890871)          NaN
8        POINT (175.672469272631 -78.72763594848804)          NaN
9       POINT (-60.68457843644514 79.69256960779688)    Greenland
10      POINT (-68.82102896321226 75.43085538451194)          NaN
11     POINT (-124.2109959246441 -6.552144275865047)          NaN
12       POINT (173.3922342988188 17.73899298314974)          NaN
13     POINT (-97.92973789686152 -19.57494428041562)          NaN
14     POI